# Toggl to Justworks conversion

This notebook explores a conversion process for Toggl hours into a format for import into Justworks.

Toggl hours are tracked daily, and we can export a report of each entry in Toggl.

Justworks expects a rollup of total hours per-person per-week.

In [ ]:
import pandas as pd

from compiler_admin.services.toggl import _prepare_input
from compiler_admin.services import files

In [ ]:
renames = {
    "Email": "Work Email",
    "First name": "First Name",
    "Hours": "Regular Hours",
    "Last name": "Last Name",
    "Start date": "Start Date"
}
df = _prepare_input("./data/toggl-sample.csv", column_renames=renames)
df.head()

In [ ]:
cols = ["Work Email", "First Name", "Last Name", "Start Date"]
df_people = df.sort_values(cols).groupby(cols, observed=False)
df_people_agg = df_people.agg({"Regular Hours": "sum"})
df_people_agg.reset_index(inplace=True)
df_people_agg.head()

In [ ]:
cols = ["Work Email", "First Name", "Last Name"]
weekly_agg = df_people_agg.groupby(cols).resample("W", label="left", on="Start Date")
weekly_agg = weekly_agg["Regular Hours"].sum().round(1).reset_index()
weekly_agg.head()

In [ ]:
weekly_agg["End Date"] = weekly_agg["Start Date"] + pd.Timedelta(days=6)
weekly_agg.head()

In [ ]:
files.write_csv("./data/justworks-sample.csv", weekly_agg, columns=["First Name", "Last Name", "Work Email", "Start Date", "End Date", "Regular Hours"])